<h1>SSLM - Super Small Language Model</h1>

<p>Este é um tutorial muito básico sobre como criar um LLM (Large Language Model).<br>
Como não é possível criar um LLM em uma máquina comum, sem GPU principalmente, iremos criar um modelo sem ambição
de ser o melhor ou que seja utilizável. Nosso modelo servirá apenas como objeto de estudo.</p>

<h2>Etapas</h2>
<ol>
    <li>Definição de escopo</li>
    <li>Definição de dataset</li>
    <li>Definição de arquitetura</li>
    <li>Treinamento</li>
    <li>Testes</li>
</ol>

<h3 style="color: blue">1. Definição de escopo</h3>
<p>A definição do escopo do nosso modelo é muito importante. Ela definirá como iremos trabalhar daqui pra frente.
Existem modelos que geram texto, classificam imagens, criam imagens, criam audio e muito mais!
Não há um modelo que seja menos complexo de desenvolver, entretanto.<br>Para nosso projeto, iremos criar um SSLM que gera texto, como ChatGPT,
 Copilot e outros famosos.</p>
 <br>
 <br>
 <p>ESCOPO DO <strong>SSLM</strong>: <b>Geração de texto</b></p>

<h3 style="color: blue">2. Definição de dataset</h3>
<p>O dataset é a base de treinamento de qualquer modelo. Um dataset de um modelo que gera imagens é um conjunto de matrizes de imagens que já existem. O dataset de um modelo que gera texto é texto. Como um computador não entende o que é uma imagem, texto, audio etc., apenas números (e em formatação binária física), é importante que transformemos esses datasets em números. As linguagens de programação já fazem a parte de transformar números lógicos em sinais elétricos binários. Nossa ideia é apenas transformar textos em números.</p>
 <br>
 <p>Para treinar um LLM como o GPT, é necessário um número imenso de texto de treinamento. A OpenAI fez aquisições de livros, revistas, artigos, 
 sites, repositórios públicos de código (GitHub e GitLab), posts públicos em redes sociais, transcrições de vídeos e muitas outras fontes ricas
 de texto para treinar o GPT. Para nosso exemplo, vamos utilizar apenas um texto grande, mas não demais, para que o computador não trave em sua execução.</p>

<h3 style="color=green">Dataset que utilizaremos: Cumprimentos</h3>
<p>Esse dataset contém mensagens que correspondem a cumprimentos não necessariamente formais entre pessoas.
Nosso SSLM deverá ser capaz de responder a mensagens simples como essas.</p>
<p>
Olá! Como você está?<br>
Oi! Tudo bem com você?<br>
Bom dia! Espero que você tenha um ótimo dia.<br>
Boa tarde! Como posso te ajudar hoje?<br>
Boa noite! Espero que você esteja bem.<br>
Oi! É um prazer falar com você.<br>
Olá! Em que posso te ajudar?<br>
Oi! Espero que você esteja tendo um bom dia.<br>
Olá! Como posso ser útil para você hoje?<br>
Oi! Que bom te ver por aqui.</p>

In [ ]:
with open ("dataset.txt", "r", encoding="utf-8") as file:
    file = file.readlines()

train_dataset = []
for line in file:
    train_dataset.append(line.replace("\n",""))

In [ ]:
train_dataset

<h3 style="color: blue">3. Definição de arquitetura</h3>
<p>O treinamento do modelo envolve várias etapas fundamentais para preparar os dados e construir um sistema capaz de gerar respostas coerentes. Primeiramente, as frases do dataset são tokenizadas, ou seja, divididas em palavras ou tokens individuais. Cada token recebe um identificador único, que é armazenado em um índice para facilitar o mapeamento entre palavras e IDs.</p>

<p>Após a tokenização, as frases são convertidas em sequências de IDs, representando cada token por seu identificador correspondente. Essas sequências são então padronizadas para um comprimento fixo utilizando padding (adicionando valores padrão no final das sequências mais curtas) ou truncamento (removendo tokens excedentes das sequências mais longas). Isso garante que todas as sequências tenham o mesmo tamanho, facilitando o processamento pelo modelo.</p>

<p>Uma boa definição de arquitetura é essencial para garantir que o modelo seja capaz de capturar os padrões desejados nos dados e realizar a tarefa proposta com eficiência.</p>

In [ ]:
index = {}

In [ ]:
def index_token(token : str, contador : int):
    if token not in index:
        index[token] = contador

In [ ]:
def tokenize(dataset: list):
    tokenized_dataset = []
    for sentence in dataset:
        tokenized_dataset.append(sentence.split())

    contador = 1
    for sentence in tokenized_dataset:
        for token in sentence:
            index_token(token, contador)
            contador += 1
    return tokenized_dataset

In [ ]:
tokenized_dataset = tokenize(train_dataset)

In [ ]:
tokenized_dataset

In [ ]:
index

In [ ]:
def sentences_to_ids(tokenized_dataset, index):
    dataset_ids = []
    for sentence in tokenized_dataset:
        sentence_ids = [index[token] for token in sentence]
        dataset_ids.append(sentence_ids)
    return dataset_ids

In [ ]:
dataset_ids = sentences_to_ids(tokenized_dataset, index)

In [ ]:
dataset_ids

In [ ]:
def pad_sequences(sequences, max_length, padding_value=0):
    padded_sequences = []
    for seq in sequences:
        if len(seq) < max_length:
            # Adiciona padding no final
            padded_seq = seq + [padding_value] * (max_length - len(seq))
        else:
            # Trunca a sequência
            padded_seq = seq[:max_length]
        padded_sequences.append(padded_seq)
    return padded_sequences

In [ ]:
max_length = 10

In [ ]:
padded_dataset_ids = pad_sequences(dataset_ids, max_length)

In [ ]:
padded_dataset_ids

<h3 style="color: blue">4. Treinamento</h3>
<p>Em seguida, são calculadas as probabilidades de transição entre os tokens com base nas sequências de treinamento. Para isso, o modelo analisa a frequência com que cada token é seguido por outro, gerando um índice de probabilidades normalizadas. Esse índice é essencial para que o modelo aprenda os padrões de transição entre palavras e possa gerar respostas baseadas em mensagens de entrada.</p>

<p>Por fim, o modelo utiliza essas probabilidades para gerar respostas. Quando uma mensagem é recebida, o sistema identifica o último token da mensagem e utiliza as probabilidades de transição para prever os próximos tokens, construindo uma resposta de forma iterativa. Esse processo continua até que um token de término seja alcançado ou que a resposta atinja um comprimento máximo predefinido.</p>

<p>Essas etapas garantem que o modelo seja capaz de compreender e responder de forma contextualizada, utilizando os padrões aprendidos durante o treinamento.</p>

In [ ]:
from collections import defaultdict

In [ ]:
def calculate_token_probabilities(sequences):
    transition_counts = defaultdict(lambda: defaultdict(int))
    transition_probabilities = {}

    # Contar transições entre tokens
    for seq in sequences:
        for i in range(len(seq) - 1):
            current_token = seq[i]
            next_token = seq[i + 1]
            transition_counts[current_token][next_token] += 1

    # Calcular probabilidades normalizadas
    for token, next_tokens in transition_counts.items():
        total_transitions = sum(next_tokens.values())
        transition_probabilities[token] = {
            next_token: count / total_transitions
            for next_token, count in next_tokens.items()
        }

    return transition_probabilities

In [ ]:
token_probabilities = calculate_token_probabilities(padded_dataset_ids)

In [ ]:
token_probabilities

In [ ]:
import random

In [ ]:
def generate_response(message, index, token_probabilities):
    # Tokenizar a mensagem e verificar se a última palavra está no índice
    tokens = message.split()
    last_word = tokens[-1] if tokens else None

    if last_word not in index:
        return "Desculpe, não entendi sua mensagem."

    # Obter o ID do último token
    current_token_id = index[last_word]
    response_ids = []

    # Gerar a resposta com base nas probabilidades
    while current_token_id != 0:  # Termina quando o token 0 é alcançado
        response_ids.append(current_token_id)
        next_token_probs = token_probabilities.get(current_token_id, {})

        if not next_token_probs:
            break  # Se não houver transições, encerra a geração

        # Escolher o próximo token com base nas probabilidades
        next_token_id = random.choices(
            list(next_token_probs.keys()),
            weights=list(next_token_probs.values())
        )[0]
        current_token_id = next_token_id

    # Converter os IDs gerados de volta para palavras
    reverse_index = {v: k for k, v in index.items()}
    response_tokens = [reverse_index[token_id] for token_id in response_ids]

    return " ".join(response_tokens)


<h3 style="color: blue">5. Testes</h3>

In [ ]:
user_message = "Olá!"
response = generate_response(user_message, index, token_probabilities)
print(f"Usuário: {user_message}")
print(f"Modelo: {response}")

<h3 style="color: blue">6. Refinamento</h3>

In [ ]:
from collections import defaultdict

In [ ]:
def calculate_ngram_probabilities(sequences, n=2):
    transition_counts = defaultdict(lambda: defaultdict(int))
    transition_probabilities = {}

    # Contar transições entre n-gramas
    for seq in sequences:
        for i in range(len(seq) - n + 1):
            ngram = tuple(seq[i:i + n - 1])  # Contexto (n-1 tokens)
            next_token = seq[i + n - 1]  # Próximo token
            transition_counts[ngram][next_token] += 1

    # Calcular probabilidades normalizadas
    for ngram, next_tokens in transition_counts.items():
        total_transitions = sum(next_tokens.values())
        transition_probabilities[ngram] = {
            next_token: count / total_transitions
            for next_token, count in next_tokens.items()
        }

    return transition_probabilities

In [ ]:
def generate_response_with_context(message, index, token_probabilities, n=2):
    tokens = message.split()
    if not tokens:
        return "Desculpe, não entendi sua mensagem."

    # Obter o contexto inicial (n-1 últimos tokens)
    context = tuple(tokens[-(n - 1):]) if len(tokens) >= n - 1 else tuple(tokens)
    context_ids = [index.get(token, None) for token in context]

    if None in context_ids:
        return "Desculpe, não entendi sua mensagem."

    response_ids = list(context_ids)

    # Gerar a resposta com base nas probabilidades de n-gramas
    while True:
        current_ngram = tuple(response_ids[-(n - 1):])  # Últimos n-1 tokens
        next_token_probs = token_probabilities.get(current_ngram, {})

        if not next_token_probs:
            break  # Se não houver transições, encerra a geração

        # Escolher o próximo token com base nas probabilidades
        next_token_id = random.choices(
            list(next_token_probs.keys()),
            weights=list(next_token_probs.values())
        )[0]
        response_ids.append(next_token_id)

        # Termina se o token gerado for o de padding (0)
        if next_token_id == 0:
            break

    # Converter os IDs gerados de volta para palavras
    reverse_index = {v: k for k, v in index.items()}
    response_tokens = [reverse_index[token_id] for token_id in response_ids if token_id in reverse_index]

    return " ".join(response_tokens)

In [ ]:
token_probabilities = calculate_ngram_probabilities(dataset_ids, n=2)

In [ ]:
# Exemplo de uso
user_message = "Oi! Tudo bem? Eu te desjo um bom dia!"
response = generate_response_with_context(user_message, index, token_probabilities, n=2)
print(f"Usuário: {user_message}")
print(f"Modelo: {response}")

In [ ]:
def learn_from_message(message, index, tokenized_dataset, dataset_ids, token_probabilities, n=2):
    #Salvar nova mensagem no dataset para aprendizado fixo
    with open ("dataset.txt", "a", encoding="utf-8") as file:
        file.write(f"{message}\n")
    
    # Tokenizar a nova mensagem
    new_tokens = message.split()
    
    # Atualizar o índice com novos tokens
    contador = max(index.values(), default=-1) + 1
    for token in new_tokens:
        if token not in index:
            index[token] = contador
            contador += 1

    # Atualizar o conjunto de dados tokenizado
    new_sentence_ids = [index[token] for token in new_tokens]
    tokenized_dataset.append(new_tokens)
    dataset_ids.append(new_sentence_ids)

    # Recalcular as probabilidades de transição
    token_probabilities.update(calculate_ngram_probabilities(dataset_ids, n=n))

In [ ]:
new_message = "carros são automóveis movidos a combustível"
learn_from_message(new_message, index, tokenized_dataset, dataset_ids, token_probabilities, n=2)

In [ ]:
# Testar a geração de resposta após o aprendizado
user_message = "O que são carros?"
response = generate_response_with_context(user_message, index, token_probabilities, n=2)
print(f"Usuário: {user_message}")
print(f"Modelo: {response}")